# 데이터 확인

In [24]:
# 데이터 로드
import pandas as pd
import pickle
# train = pd.read_table("data/ratings_train.txt")
# test = pd.read_table("data/ratings_test.txt")

In [25]:
with open('preprocess_형태소_음절.pkl', 'rb') as file: # 파일을 바이너리 읽기 모드(rb)로 열기
    train = pickle.load(file)
    test = pickle.load(file)

In [26]:
train

AttributeError: 'DataFrame' object has no attribute '_data'

In [5]:
# 데이터 개수 확인
print(len(train), len(test))

# 데이터 구성 확인
train

AttributeError: 'DataFrame' object has no attribute '_data'

# 데이터 정제

In [ ]:
# 결측치(nan) 확인 == nan의 개수 있는지 확인
print(train.isnull().sum())
print("\n")
print(test.isnull().sum())

In [ ]:
# 결측치 제거
train = train.dropna()
test = test.dropna()

In [ ]:
# 결측치(nan) 확인 == nan의 개수 확인
print(train.isnull().sum())
print("\n")
print(test.isnull().sum())

In [ ]:
# 한글 외에 다른 요소들 없애기
train["document"] = train["document"].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', "")
test["document"] = test["document"].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', "")

In [ ]:
# 전처리를 진행하면서 공백이 생겼다면 이를 nan으로 설정하고 다시 결측치를 제거하는 작업을 해줍니다.
import numpy as np

train['document'].replace('', np.nan, inplace=True)
test['document'].replace('', np.nan, inplace=True)

print(train.isnull().sum()) # 다시 한번 결측치가 있는지 확인
print(test.isnull().sum()) # 다시 한번 결측치가 있는지 확인

In [ ]:
# 결측치 제거
train = train.dropna()
test = test.dropna()

# 데이터 전처리

In [ ]:
from tqdm import tqdm # for문의 실행되는 정도를 확인할 수 있다.
from konlpy.tag import Kkma # 형태소 분석을 위함

kkma = Kkma()

## 형태소 단위로 자르기

In [ ]:
a = []
for i,value in tqdm(enumerate(train["document"])):
    try:
        a.append(str(" ".join([k[0] for k in kkma.pos(value)])))
    except:
        a.append("")

In [ ]:
b = []
for i,value in tqdm(enumerate(test["document"])):
    try:
        b.append(str(" ".join([k[0] for k in kkma.pos(value)])))
    except:
        b.append("")

## 음절 단위로 자르기

In [ ]:
c = []
for i,value in tqdm(enumerate(train["document"])):
    try:
        c.append(str(" ".join([j for i in value.split() for j in i])))
    except:
        c.append("")

In [ ]:
d = []
for i,value in tqdm(enumerate(test["document"])):
    try:
        d.append(str(" ".join([j for i in value.split() for j in i])))
    except:
        d.append("")

## train, test 데이터에 열 추가

In [ ]:
train["document_morpheme"] = a
test["document_morpheme"] = b
train["document_syllable"] = c
test["document_syllable"] = d

In [ ]:
train.head()

# 데이터 다시 정제하기

In [ ]:
# 데이터가 비어있으면 해당 행을 모두 제거해줍니다.

train = train[train["document_morpheme"]!= ""]
test = test[test["document_morpheme"]!= ""]

train = train[train["document_syllable"]!= ""]
test = test[test["document_syllable"]!= ""]

# 만들어진 train, test 객체를 저장

In [ ]:
# 만들어진 train, test데이터 객체 저장 // 추 후에 load해서 빠르게 사용하기 위함

import pickle

In [ ]:
# with open('preprocess_형태소_음절.p', 'wb') as file: # 파일을 바이너리 쓰기 모드(wb)로 열기
#     pickle.dump(train, file)
#     pickle.dump(test, file)

In [ ]:
with open('preprocess_형태소_음절.p', 'rb') as file: # 파일을 바이너리 읽기 모드(rb)로 열기
    train = pickle.load(file)
    test = pickle.load(file)

In [ ]:
len(train), len(test)

# 정수 인코딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(train["document_morpheme"])

# 벡터화
x_train = tokenizer.texts_to_sequences(train["document_morpheme"])
x_test = tokenizer.texts_to_sequences(test["document_morpheme"])

# 총 단어 수
total_word = len(tokenizer.word_index)

# 단어 집합의 크기
vocab_size = total_word+1

#
import numpy as np
y_train = np.array(train["label"])
y_test = np.array(test["label"])

In [ ]:
# 데이터의 최대 길이 확인

# train + test 데이터 합치기
all_data = pd.concat([train, test], axis = 0)

max_len = 0
for i in all_data["document"]:
    if max_len <= len(i):
        max_len = len(i)
        
print(max_len)

In [ ]:
# 패딩을 통해 길이를 맞춰줍니다.
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape, max_len, vocab_size

# 모델1 - 기본 dense layer

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

### 모델 구축

In [ ]:
model__ = Sequential()
model__.add(Dense(64, activation='relu', input_shape=(140,)))
model__.add(Dropout(0.5))
model__.add(Dense(32, activation='relu'))
model__.add(Dense(1, activation='sigmoid'))

model__.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model__.summary()

### 모델 학습

In [ ]:
model__.fit(x_train, y_train, epochs = 10, batch_size = 64, validation_split = 0.2)

### 그래프

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# epoch마다 정확도와 validation 정확도의 수치를 그래프로 표현해봅니다.
plt.title("Accuracy")

plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history.history["val_acc"], color="r", label="validation")

plt.gca().invert_yaxis() # y축 반대로 뒤집기
plt.show()

In [ ]:
plt.title("Loss")

plt.plot(history.history["loss"], color="g", label="train")
plt.plot(history.history["val_loss"], color="r", label="validation")

plt.show()

### 모델 2 - lstm

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, 100))
model.add(LSTM(140))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=15, batch_size=64, validation_split=0.2)


In [ ]:
print(history.history.keys())

### 그래프

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# epoch마다 정확도와 validation 정확도의 수치를 그래프로 표현해봅니다.
plt.title("Accuracy")

plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history.history["val_acc"], color="r", label="validation")

plt.gca().invert_yaxis() # y축 반대로 뒤집기
plt.show()

In [ ]:
plt.title("Loss")

plt.plot(history.history["loss"], color="g", label="train")
plt.plot(history.history["val_loss"], color="r", label="validation")

plt.show()

# 모델 3 - cnn + bidirectional_lstm

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, Dropout, Conv1D, \
                            MaxPooling1D, Bidirectional, LSTM, Activation

from tensorflow.keras.models import Sequential

### 모델 구축

In [ ]:
model_ = Sequential()

model_.add(Embedding(vocab_size,100))
model_.add(Dropout(0.4))
model_.add(Conv1D(300, 3, padding= 'same', activation='relu', strides=1))
model_.add(MaxPooling1D(pool_size=4))
model_.add(Bidirectional(LSTM(100, activation = "relu")))
model_.add(Dropout(0.5))
model_.add(Dense(32, activation="relu"))
model_.add(Dense(1))
model_.add(Activation('sigmoid'))

model_.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
model_.summary()

### 모델 학습

In [ ]:
history_ = model_.fit(x_train, y_train, epochs=15, batch_size=64, 
                      validation_split=0.2)


### 그래프

In [ ]:
print(history_.history.keys())

In [ ]:
# epoch마다 정확도와 validation 정확도의 수치를 그래프로 표현해봅니다.
plt.title("Accuracy")

plt.plot(history_.history["accuracy"], color="g", label="train")
plt.plot(history_.history["val_accuracy"], color="r", label="validation")

plt.gca().invert_yaxis() # y축 반대로 뒤집기
plt.show()

In [ ]:
plt.title("Loss")

plt.plot(history_.history["loss"], color="g", label="train")
plt.plot(history_.history["val_loss"], color="r", label="validation")

plt.show()

In [ ]:
# loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (model_.evaluate(x_test, y_test)[1]))

In [ ]:
def sentiment_predict(new_sentence):
    new_sentence = kkma.morphs(new_sentence) # 토큰화
#     new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:

sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
history__ = model_.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:
# 정확도와 손실도 를 그림으로 나타내줘 epoch마다 정확도와 validation 정확도의 수치를 눈으로 확인하게끔 해줍니다.
plt.title("Episodic Memory Q & A Accuracy")
# plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history__.history["accuracy"], color="g", label="train")
# plt.plot(history.history["val_acc"], color="r", label="validation")
plt.plot(history__.history["val_accuracy"], color="r", label="validation")
plt.legend(loc="best")

plt.show()

In [ ]:
# loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도:", model__.evaluate(x_test, y_test))

In [ ]:
# 정수 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()


tokenizer.fit_on_texts(train["document_first"])


x_train = tokenizer.texts_to_sequences(train["document_first"])
x_test = tokenizer.texts_to_sequences(test["document_first"])


# 총 단어 수
total_word = len(tokenizer.word_index)

# 단어 집합의 크기
vocab_size = total_word+1

y_train = np.array(train["label"])
y_test = np.array(test["label"])

# 패딩을 통해 길이를 맞춰줍니다.
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, 100))
model.add(LSTM(140))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_split=0.2)


In [ ]:
# 정확도와 손실도 를 그림으로 나타내줘 epoch마다 정확도와 validation 정확도의 수치를 눈으로 확인하게끔 해줍니다.
plt.title("Episodic Memory Q & A Accuracy")
# plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history_.history["accuracy"], color="g", label="train")
# plt.plot(history.history["val_acc"], color="r", label="validation")
plt.plot(history_.history["val_accuracy"], color="r", label="validation")
plt.legend(loc="best")

plt.show()

In [ ]:
# loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)))

In [ ]:
import string
string.punctuation

In [ ]:
model.summary()

In [ ]:
model__.fit